In [ ]:
#### import global modules
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from yaml import safe_load
import google.oauth2.credentials
from google.cloud import bigquery
import gc

# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_queries = pth_project / 'core' / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
sys.path.insert(0, str(pth_project))
d_project_config = safe_load(pth_creds.open())
# d_params = safe_load((pth_project / 'core' / 'parameters' / 'common.yaml').open())['data_extract']

# import local modules
from core.utils.gcp import connect_bq_services
# from core.etl.extract import extract_bq_data, extract_pr_codes, format_conv_df, filter_convs

# Connect to google services
bq_client = connect_bq_services(d_project_config['gcp-project-name'])
pd.options.display.max_rows = 100

In [ ]:
def extract_bq_data(bq_client, sql=None, pth_query=None):
    if sql is not None:
        df = bq_client.query(sql).to_dataframe()
    elif pth_query is not None:
        sql = pth_query.read_text()
        df = bq_client.query(sql).to_dataframe()
    else:
        raise ValueError('`sql` or `pth_query` should be set')  
    return df

In [ ]:
Customer_details='''


  --Take LAST day SNAPSHOT FOR active customers -- remove the eligibilty criteria --AND use it AS contract start feature
  
DECLARE
  start_dt_snpsht DATE DEFAULT '2022-12-01';
DECLARE
  end_dt_snpsht DATE DEFAULT '2023-02-28';
WITH
  date_sql AS (
  SELECT
    DATE_TRUNC(DATE_SUB(start_dt_snpsht, INTERVAL 3 month), month) AS eligibility_date ),
  
  ADC_Customer_Base AS (
  SELECT
    dealer_customer_id,
    customer_id,
    primary_login_id,
    dealer_name,
    DATE(join_date) AS join_date,
    account_type_name,
    customer_type_name,
    CASE
      WHEN account_type_name='Standalone' THEN 'Smart_Camera'
      WHEN account_type_name='Awareness and Automation' THEN 'Smart_Automation_Plus'
    ELSE
    'Monitored'
  END
    AS Package,
    DATE_DIFF(end_dt_snpsht,DATE(join_date), MONTH) AS Tenure_months,
    1 as Interactivity_flag #Interactivity=Having ADC account

  FROM
    `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_account_details`
  WHERE
    DATE(last_updt_ts) <= end_dt_snpsht
    AND DATE(last_updt_ts) >= start_dt_snpsht
    AND dealer_name in ('TELUS Communications Inc.')
    QUALIFY ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY last_updt_ts DESC) = 1
  ORDER BY
    join_date,
    customer_id ),
  
Telus_customers AS (
  SELECT
    cust_bus_cust_id as Telus_ID,
    cast (bacct_bus_bacct_num as STRING) AS BAN,
    DATE(pi_cntrct_start_ts) AS contract_start_date,
    DATE(pi_cntrct_end_ts) AS contract_end_date,
    DATE(pi_actvn_ts) as Intial_activation_date,
    CASE
      WHEN DATE(pi_cntrct_end_ts) = "9999-12-31" THEN 'contract_expired'
      WHEN DATE_DIFF(DATE(pi_cntrct_end_ts),end_dt_snpsht, DAY) < 0 THEN 'contract_expired'
      WHEN DATE_DIFF(DATE(pi_cntrct_end_ts),end_dt_snpsht, DAY) <= 90 THEN 'contract_expiring_within_90_days'
      WHEN DATE_DIFF(DATE(pi_cntrct_end_ts),end_dt_snpsht, DAY) > 90 THEN 'contract_expiring_after_90_days'
    ELSE
    'Undefined_contract_end_date'
  END
    AS contract_end_status,
    DATE_DIFF(end_dt_snpsht,DATE(pi_cntrct_start_ts), MONTH) AS Contract_start_months
  FROM
    `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht`
  WHERE
    DATE(prod_instnc_ts) = end_dt_snpsht
    AND pi_prod_instnc_typ_cd ='SMHM' #Serice type
    AND bus_prod_instnc_src_id = 1001 #BANs that are FOR home services
    AND pi_prod_instnc_stat_cd IN ('A')
    AND consldt_cust_typ_cd = 'R' QUALIFY ROW_NUMBER() OVER (PARTITION BY cust_bus_cust_id ORDER BY prod_instnc_ts DESC) = 1
  ORDER BY
    cust_bus_cust_id ),
    
ADT_migrated_customers as (

SELECT distinct udf4 as BAN
      ,cust_no
      ,co_no
      ,branch_no
      ,custype_id
      ,custstat_id
      , 1 as ADT_migrated_customer
      

  FROM `cio-datahub-enterprise-pr-183a.src_mastermind.bq_customer`
  where branch_no in (999100,999500)
  and udf4 is not NULL


),

Merge_data AS (
  SELECT
    a.*,b.*,c.ADT_migrated_customer
  FROM
    Telus_customers a
  LEFT JOIN
    ADC_Customer_Base b
  ON
    a.Telus_ID=b.dealer_customer_id
LEFT JOIN
ADT_migrated_customers c
on a.BAN=c.BAN)

SELECT
  *
FROM
  Merge_data
  
  
'''

In [ ]:
Customer_info=extract_bq_data(bq_client, sql=Customer_details)

In [ ]:
Customer_info.info()

In [ ]:
Customer_info.isna().sum()*100/Customer_info.shape[0]

In [ ]:
Customer_info['Package'].fillna('Unknown_Package',inplace=True)
# Customer_info['Tenure_months'].fillna('Unknown_Tenure',inplace=True)
Customer_info['dealer_name'].fillna('Unknown_Dealer',inplace=True)
Customer_info['Interactivity_flag'].fillna(0,inplace=True)
Customer_info['ADT_migrated_customer'].fillna(0,inplace=True)

In [ ]:
Customer_info['contract_start_date'].value_counts()

In [ ]:
Customer_info['contract_end_status'].value_counts()

In [ ]:
Customer_info['dealer_name'].value_counts()

In [ ]:
Customer_info['ADT_migrated_customer'].value_counts()

In [ ]:
Customer_info['ADT_migrated_customer'].value_counts(normalize=True)*100

In [ ]:
Customer_info['Interactivity_flag'].value_counts(normalize=True)*100

In [ ]:
pd.DataFrame(Customer_info.groupby(['ADT_migrated_customer','Interactivity_flag']).agg(
    
    Customer_count= ('Telus_ID','nunique'),
    Activation_date=('Intial_activation_date','min')
    # Email_Opened_count=('OPENED','sum'),
    # CT_Opened_count=('CLICKTHROUGH','sum'),
    # softbounce_count=('SOFTBOUNCE','sum'),
    # Hardbounce_count=('HARDBOUNCE','sum'),
    # # Unsub_count=('UNSUBSCRIBE','sum'),
    # campaign_date_min=('Campaign_date','min'),
    # campaign_date_max=('Campaign_date','max')

    
    # # Customer_Share= ('customer_id',lambda x:x.count()*100/Merge_DF_4.shape[0])
    # Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
    # Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)
).reset_index())

In [ ]:

Customer_info[Customer_info['BAN']=='605508523']

In [ ]:
Customer_info['Interactivity_flag'].value_counts(normalize=True)*100

In [ ]:
# ADT_migrated_customers=Customer_info[(Customer_info['ADT_migrated_customer']==1) & (Customer_info['contract_end_status']=='contract_expired') ]
# [['Telus_ID','BAN','dealer_name']]

In [ ]:
# ADT_migrated_customers.info()

In [ ]:
# ADT_migrated_customers.head()[['Telus_ID','BAN']]

In [ ]:
# ADT_migrated_customers['dealer_name'].value_counts()

In [ ]:
Customer_info['Package'].value_counts()

In [ ]:
BAN_list=Customer_info['BAN'].value_counts().rename_axis('BAN').reset_index(name='unique_counts')

In [ ]:
BAN_list

In [ ]:
BAN_list_1=BAN_list[BAN_list.unique_counts==1]

In [ ]:
BAN_list_1.info()

In [ ]:
Customer_info_1=Customer_info.merge(BAN_list_1[['BAN']],on='BAN',how='inner')

In [ ]:
Customer_info_1['BAN'].value_counts().rename_axis('BAN').reset_index(name='unique_counts')

In [ ]:
Customer_info_1.info()

In [ ]:
Customer_info_1.head()

In [ ]:
Customer_info_1['Contract_start_months'].value_counts()

In [ ]:
Customer_info_1['Tenure_months'].value_counts()

In [ ]:
# Customer_info_1[Customer_info_1.Tenure_months==0]

In [ ]:
pd.DataFrame(Customer_info_1.groupby(['ADT_migrated_customer','Interactivity_flag']).agg(
    
    Customer_count= ('Telus_ID','nunique'),
    # Email_Opened_count=('OPENED','sum'),
    # CT_Opened_count=('CLICKTHROUGH','sum'),
    # softbounce_count=('SOFTBOUNCE','sum'),
    # Hardbounce_count=('HARDBOUNCE','sum'),
    # # Unsub_count=('UNSUBSCRIBE','sum'),
    # campaign_date_min=('Campaign_date','min'),
    # campaign_date_max=('Campaign_date','max')

    
    # # Customer_Share= ('customer_id',lambda x:x.count()*100/Merge_DF_4.shape[0])
    # Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
    # Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)
).reset_index())

In [ ]:
config= bigquery.job.LoadJobConfig()

# config._properties['timePartitioning'] = {'field': 'Month_Year'}
config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

Table_BQ = 'SHS.SHS_churn_model_base_trn_data_dec22_feb23'

bq_table_instance= bq_client.load_table_from_dataframe(Customer_info_1, Table_BQ,job_config=config)